In [1]:
import numpy as np
import pandas as pd
import torch_geometric
import networkx as nx
from torch_geometric.data import Data, Dataset
from torch_geometric.datasets.planetoid import Planetoid
from torch_geometric.transforms.to_undirected import ToUndirected
import torch

In [4]:
data, splits_edge = torch.load("/home/jrm28/fairness/graphair/fairgraph/method/checkpoint/out/AUGMENTED_facebook_10000_epochs_2024-03-13_12-21-55/splits.pt", map_location=torch.device('cpu'))

## The "fair" graph generated by GraphAir is **much** denser than the original graph

In [7]:
data.edge_index.shape

torch.Size([2, 174290])

In [11]:
facebook = data
y = data.y

In [28]:
transform = ToUndirected()
data = transform(data)

In [32]:
facebook.edge_index = data.edge_index[:, data.edge_index[0] <= data.edge_index[1]]

## EDA

In [34]:
all_possible_males = ((y == 0).sum()**2)/2
all_possible_females = ((y == 1).sum()**2)/2
all_possible_male_female = ((y == 0).sum() * (y == 1).sum()) / 2

print("Num edges in each group - Num nodes in the group")
print("MM", (y[facebook.edge_index].sum(0) == 0).sum(), '-', (y == 0).sum())
print("MF", (y[facebook.edge_index].sum(0) == 1).sum())
print("FF", (y[facebook.edge_index].sum(0) == 2).sum(), '-', (y == 1).sum())


print("MM", (y[facebook.edge_index].sum(0) == 0).sum() / all_possible_males)
print("MF", (y[facebook.edge_index].sum(0) == 1).sum() / all_possible_male_female)
print("FF", (y[facebook.edge_index].sum(0) == 2).sum() / all_possible_females)

Num edges in each group - Num nodes in the group
MM tensor(36671) - tensor(517)
MF tensor(40017)
FF tensor(11816) - tensor(275)
MM tensor(0.2744)
MF tensor(0.5629)
FF tensor(0.3125)


## Ratio MF / (FF + MM)

In [35]:
(y[facebook.edge_index].sum(0) == 1).sum() / ((y[facebook.edge_index].sum(0) == 2).sum() + (y[facebook.edge_index].sum(0) == 0).sum())

tensor(0.8253)